## Automated Speaker Verification - A Study in Deep Neural Networks and Transfer Learning 

*"Identity theft is not a joke" - Dwight Shrute, The Office*

Automated speaker verification (ASV), in the modern day, is omnipresent in smart devices and in services offered by call centres. It serves as a biometric means of authenticating a claimed identity by analysing a spoken sample of their voice. 
our team sought to extend our understanding of neural style transfer by exploring how convolutional neural networks could be used to harness the ability to speak in the voice of another person.

#### Requirements
TensorFlow needs to be installed before running the training script. Code is tested on TensorFlow version 1.0.1 for Python 2.7 and Python 3.5.

In addition, librosa must be installed for reading and writing audio.

To install the required python packages, run

```shell
pip install -r requirements.txt
```
For GPU support, use
```shell
pip install -r requirements_gpu.txt
```

#### Resources

https://www.deepmind.com/blog/wavenet-a-generative-model-for-raw-audio

https://becominghuman.ai/convoice-real-time-zero-shot-voice-style-transfer-with-convolutional-network-4c7b7fff66c9

https://arxiv.org/pdf/1703.10135.pdf

https://arxiv.org/pdf/1905.05879v2.pdf

https://medium.com/@ageitgey/machine-learning-is-fun-part-6-how-to-do-speech-recognition-with-deep-learning-28293c162f7a



#### A Brief Intro to Text-to-Speech Synthesis (TTS)

-- SOME TEXT --

#### Network Architecture:

Connectionist temporal classification loss (CTC loss)
